### 1) What are the main motivations for reducing a dataset’s dimensionality? - 0.5 points
### What are the main drawbacks? - 0.5 points

### Load the MNIST dataset (given below) 


In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
print(mnist.data.shape)
print(mnist.target.shape)

(70000, 784)
(70000,)


### 2) Split it into a training set and a test set
### Take the first 60,000 instances for training, and the remaining 10,000 for testing. - 1 point

In [14]:
from sklearn.preprocessing import StandardScaler

data = StandardScaler().fit_transform(mnist.data)
X_train = data[:-10000,:]
X_test = data[-10000:,:]
print(X_train.shape)
print(X_test.shape)

y_train = mnist.target[:-10000]
y_test = mnist.target[-10000:]
print(y_train.shape)
print(y_test.shape)

(60000, 784)
(10000, 784)
(60000,)
(10000,)


### 3) Train a Random Forest classifier on the dataset and time how long it takes, - 1 point
### then evaluate the resulting model on the test set. - 1 point

In [17]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=123456)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=123456)

This model takes 01:29.70
After StandardScaler, it takes02:55.25

In [18]:
from sklearn.metrics import accuracy_score
predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)
print(f'accuracy score: {accuracy:.3}')

accuracy score: 0.97


### 4) Next, use PCA to reduce the dataset’s dimensionality, with an explained variance ratio of 95%. - 2 + 2 points

In [20]:
from sklearn.decomposition import PCA
import pandas as pd 

data = StandardScaler().fit_transform(mnist.data)

pca=PCA(n_components=0.95)
pca_data=pca.fit_transform(data)
df_pca_data=pd.DataFrame(pca_data)

print(df_pca_data.shape)
print(pca.explained_variance_ratio_)

(70000, 332)
[0.05642719 0.04041226 0.03738273 0.02893    0.02520752 0.02192549
 0.01914282 0.01740684 0.01532232 0.01396087 0.01342175 0.01201421
 0.01113962 0.01090582 0.01027986 0.00994955 0.00931255 0.00919635
 0.008886   0.00863195 0.00821741 0.00798417 0.00762573 0.00742315
 0.0071657  0.00689314 0.00681399 0.00654588 0.00627293 0.00610345
 0.00597261 0.00589304 0.00567358 0.00559358 0.00552473 0.00534443
 0.00527593 0.00515841 0.00505498 0.00477438 0.00476312 0.00465155
 0.00453454 0.00445757 0.00442313 0.00437877 0.00437294 0.00427724
 0.00424808 0.00418524 0.00404059 0.00396258 0.00393176 0.00390562
 0.00386444 0.00377501 0.00373883 0.00368328 0.00360377 0.0035637
 0.00349289 0.00344527 0.00343239 0.00341    0.00334463 0.00332107
 0.00329803 0.00319433 0.0031711  0.00315431 0.00309941 0.00305782
 0.00305095 0.0030396  0.00296635 0.00292958 0.00291295 0.00290232
 0.00288608 0.00287022 0.00284514 0.00281524 0.00279245 0.00278629
 0.00278233 0.00276733 0.0027542  0.00272901 0.002

In [21]:
import numpy as np
print(np.sum(pca.explained_variance_ratio_))

0.9500311796713796


In [22]:
X_train_pca = df_pca_data.iloc[:-10000,:]
X_test_pca = df_pca_data.iloc[-10000:,:]
print(X_train_pca.shape)
print(X_test_pca.shape)

(60000, 332)
(10000, 332)


### 5)  Train a new Random Forest classifier on the reduced dataset and see how long it takes. - 1 point
### Was training much faster? 

In [23]:
rf = RandomForestClassifier(n_estimators=100, random_state=123456)
rf.fit(X_train_pca, y_train)

RandomForestClassifier(random_state=123456)

This model takes 03:01.81. after StandardScaler, it takes 03:47.30
When I used PCA, the model is much slower than above model without PCA.

### 6) Next evaluate the classifier on the test set: how does it compare to the previous classifier? - 1 point

In [24]:
from sklearn.metrics import accuracy_score
predicted = rf.predict(X_test_pca)
accuracy = accuracy_score(y_test, predicted)
print(f'accuracy score: {accuracy:.3}')

accuracy score: 0.938


From previous model without using PCA, I got accuracy 0.97 with 784 features, and took 02:55. From second model with using PCA, I got 0.938 with 332 features, and took 03:47. 
Apparently, The feature reduction via PCA is useful to speed up to compute and have good accuracy. However, in the above case, using PCA could cause much slower and less accurcay even though there is significant reduction of data dimentionality. I guess reason why PCA is not helping to improve model is that features either have non-linear relationships or no relationships at all. Also, the performance could vary depending on the random forest.  I think we have to take into consideration using PCA according to "If the variables are correlated, PCA can achieve dimension reduction. If not, PCA just orders them according to their variances". PCA is not always good.